# JSON to Data conversion
## Author: [Jeremiah Croshaw](https://linktr.ee/jeremiahcroshaw)
#### Last Edited: Sept 23 2020

Since this code was written while employed by [Quantum Silicon Inc.](https://www.quantumsilicon.com/), I have been advised to share it under the GNU-GPL
***
Copyright (C) 2020  Jeremiah Croshaw

This program is free software; you can redistribute it and/or
modify it under the terms of the GNU General Public License
as published by the Free Software Foundation; version 2
of the License.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301, USA.
***

### This code was used to convert the labelled data .JSON file (made with [labelme](http://labelme.csail.mit.edu/Release3.0/)) into useable training data for CNN training.

### This code was developed for follow up work to our [published work](https://iopscience.iop.org/article/10.1088/2632-2153/ab6d5e) on defect segmentation of scanning probe images of the H-Si(100) surface.  
The included .SXM file is a subsection of the total data set. Details regarding the upgraded data set and neural network are found in Croshaw's PhD Thesis

author corresponence: croshaw@ualberta.ca

In [1]:
import os
import json
import numpy as np
import cv2
import h5py

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion,distance_transform_edt
from scipy.ndimage import gaussian_filter

Once the defects have been extracted and converted into a binary map of an image, the euclidian distance transform is taken to approximate the centre of each labelled defect.  This series of functions uses finds each maximum in the distance transform with the assumption that each peak corresponds to the centre of the defect.  Once the centre of the defect is known, it can be cropped for future use.
Variations of these functions are used in multiple examples, [here](https://stackoverflow.com/questions/16842823/peak-detection-in-a-noisy-2d-array) is where I got it.
#### detect_peaks(image):

***
input: 
- image -this corresponds to the euclidean distance transform of a binary mask of the defects.

output: 
- detected_peaks -a mask containing only peaks

#### filter_quadratic(data,condition):
applies a filter to remove any peaks that are too close to each other (if they are within a certain distance, they are likely from the same defect)
***
input:
- data -The mask of images with the peaks marked
- condition -a reference to the following function the_condition(xs,ys) which defines an acceptable distance between peaks

output:
- result -The filtered peaks

#### the_condition(xs,ys):
used to mark an acceptable distance between two points.  If xs is too close to previous peaks (stored in ys) then a FALSE is returned and the point xs is not added to ys.
***
input:
- xs,ys - arrays of coordinates to compare 

output:
- returns a function which indicates TRUE if the condition is met, FALSE if not.

In [2]:
def detect_peaks(image):
    """
    Takes an image and detect the peaks usingthe local maximum filter.
    Returns a boolean mask of the peaks (i.e. 1 when
    the pixel's value is the neighborhood maximum, 0 otherwise)
    """

    # define an 8-connected neighborhood
    neighborhood = generate_binary_structure(2,2)

    #apply the local maximum filter; all pixel of maximal value 
    #in their neighborhood are set to 1
    local_max = maximum_filter(image, footprint=neighborhood)==image
    #local_max is a mask that contains the peaks we are 
    #looking for, but also the background.
    #In order to isolate the peaks we must remove the background from the mask.

    #we create the mask of the background
    background = (image==0)

    #a little technicality: we must erode the background in order to 
    #successfully subtract it form local_max, otherwise a line will 
    #appear along the background border (artifact of the local maximum filter)
    eroded_background = binary_erosion(background, structure=neighborhood, border_value=1)

    #we obtain the final mask, containing only peaks, 
    #by removing the background from the local_max mask (xor operation)
    detected_peaks = local_max ^ eroded_background

    return detected_peaks

def filter_quadratic(data,condition):
    result = []
    for element in data:
        if all(condition(element,other) for other in result):
            result.append(element)
    return result

def the_condition(xs,ys):
    return sum((x-y)*(x-y) for x,y in zip(xs,ys)) > 3 #change this condition if you want the acceptable peaks to be closer together or further apart.

#### Normalize(image):
just a simply normalize filter for images
***

input
- image - the image to normalize

output
- image - thie image after normalization

In [3]:
def normalize(image):
    image = (image-np.min(image))*255/(np.max(image)-np.min(image))
    return image

### Import the JSON files and create binary mask for each defect
***
imports the json_files and corresponding images and extracts the defects and contour coordinates from the .json file.  It then creates a mask of each defect label creating a set of binary arrays corresponding to each defect in the .json files.  the resulting list all_defects will have a size of (number of images,number of labels + 1,pixels_x,pixels_y).  the + 1 is the image itself.

In [43]:
#define locations where json files and images are saved.
json_folder = ".\\json_files" 
image_folder = ".\\images"

all_defects = [] # empty list used to store the binary masks for the defects.  Will end up containing the image, and a binary mask for each of the 13 defects 
defect_classes = np.array(['DB','As2','dihydride','single_dihydride','1by1','3by1','dot','raised_silicon','etch_pit','gunk','step_edge','missing_dimer','unknown'])

defects_count = np.full((defect_classes.shape),0)
defects_pixels = np.full((defect_classes.shape),0)

files = os.listdir(json_folder)

for i in range(len(files)): #going over all json folders

    print(image_folder + '\\' + files[i][:-4] + 'jpg')
    img = cv2.imread(image_folder + '\\' + files[i][:-4] + 'jpg',0) # loads the image
    defects = np.full((14,img.shape[0],img.shape[1]),0) #create an empty array for defect masks from json
    defects[0] = img #set first entry as image
    with open(json_folder + '\\' + files[i]) as f: #open the json file
        datastore = json.load(f)
    f.close()
    for x in range(len(datastore['shapes'])):
        label = datastore['shapes'][x]['label']#extract label from json file

        if label == 'raised_silcon': #spelling mistake I didn't want to fix
            index = 7
        else:
            index = int(np.argwhere(defect_classes == label))
        #print(label,": ", index)

        defects_count[index] = defects_count[index] + 1 #count the defects labelled

        if datastore['shapes'][x]['shape_type'] == 'polygon': #if it's a polygon shape, draw a polygon using cv2
            points = np.asarray(datastore['shapes'][x]['points'],int)
            cv2.fillPoly(defects[index+1],np.int32([points]),1)

        if datastore['shapes'][x]['shape_type'] == 'rectangle': #if it's a rectangle, draw a rectangle using cv2
            points = np.asarray(datastore['shapes'][x]['points'],int)
            cv2.rectangle(defects[index+1],(points[0][0],points[0][1]),(points[1][0],points[1][1]),1,-1)
    
    for x in range(len(defect_classes)): #sum over pixels here
        defects_pixels[x] = defects_pixels[x] + np.sum(defects[x+1])
    all_defects.append(defects)

all_defects = np.asarray(all_defects)
print(all_defects.shape) # should be (number of immages, number of classes + 1, pixels_x, pixels_y)

.\images\tile_scan_084_bwd.jpg
.\images\tile_scan_082_fwd.jpg
(2, 14, 1024, 1024)


## Create .h5 file from the binary masks
***
takes the full binary masks and applies operations to them (i.e. reduce size, rotate, filp) to increase the number of images available for the training set.

In [42]:
#plt.imshow(all_defects[1][1])
#plt.show()

complete_dataset = [] #complete data set is used to store the flipped, rotated, and cropped images as well.
rotation = 0
for r in range(0,1): # this loop runs over the rotation angle for each image.  Set to 4 if you want 4 90 degree rotations
    for i in range(0,all_defects.shape[0]):#loops over number of json files
        interval = 8 # this is the factor in which the size fo the images are reduced.  8 means a 1024 x 1024 image will be reduced to 64 128 x 128 images
        pixel_range = int(1024/interval)

        for j in range(0,interval): #loops over number of times we divide the image
            for k in range(0,interval):
                all_defects[i][0][j*pixel_range:(j+1)*pixel_range,k*pixel_range:(k+1)*pixel_range] = normalize(all_defects[i][0][j*pixel_range:(j+1)*pixel_range,k*pixel_range:(k+1)*pixel_range])

            #normal images
                segment = []
                for l in range(0,len(all_defects[i])): #loops over number of defects per json file
                    if l == 1: # need to create an H-Si label.  This array is 1 if all other defects are 0, the l = 0 condition puts it right after the image before the DB class
                        H_Si = np.full((pixel_range,pixel_range),1.)
                        for x in range(0,len(all_defects[i])):
                            H_Si[all_defects[i][x][j*pixel_range:(j+1)*pixel_range,k*pixel_range:(k+1)*pixel_range] == 1] = 0
                        H_Si = H_Si.tolist()
                        segment.append(np.rot90(H_Si,k=rotation))
                        segment.append(np.rot90(all_defects[i][l][j*pixel_range:(j+1)*pixel_range,k*pixel_range:(k+1)*pixel_range],k=rotation))
                        complete_dataset.append(segment)
                    else:
                        segment.append(np.rot90(all_defects[i][l][j*pixel_range:(j+1)*pixel_range,k*pixel_range:(k+1)*pixel_range],k=rotation))
                        complete_dataset.append(segment)
            """
            #################################
            #you can uncomment this section if you want to include image transposes as well
            #transpose images
                segment = []
                for l in range(0,len(all_defects[i])): #loops over number of defects per json file
                    if l == 1: # need to create an H-Si label.  This array is 1 if all other defects are 0, the l = 0 condition puts it right after the image before the DB class
                        H_Si = np.full((pixel_range,pixel_range),1.)
                        for x in range(0,len(all_defects[i])):
                            H_Si[all_defects[i][x][j*pixel_range:(j+1)*pixel_range,k*pixel_range:(k+1)*pixel_range] == 1] = 0
                        H_Si = H_Si.tolist()
                        segment.append(np.rot90(np.flip(H_Si),k=rotation))
                        segment.append(np.rot90(np.flip(all_defects[i][l][j*pixel_range:(j+1)*pixel_range,k*pixel_range:(k+1)*pixel_range]),k=rotation))
                        complete_dataset.append(segment)
                    else:
                        segment.append(np.rot90(np.flip(all_defects[i][l][j*pixel_range:(j+1)*pixel_range,k*pixel_range:(k+1)*pixel_range]),k=rotation))
                        complete_dataset.append(segment)
            #end of image transpose section
            ################################
            """
    


# More filtering can be done to the data here such as add noise, or artifically normalize images to simulate this segment taken from a surface with a step edge in the image (or upper, middle, and lower if simulating a surface with two step edges)
# These steps were only going to be done if the NN performance was poor, but were deemed unecessary after its performance was observed.

complete_dataset = np.asarray(complete_dataset,dtype = int)# you may run into memory errors here if you include the rotation and transpose as well
print("the complete dataset size: ",complete_dataset.shape)

h5f = h5py.File('.\\defect_data.h5','w')
h5f.create_dataset('complete_dataset',data = complete_dataset) # change the name here according to how the data has been filtered.
h5f.close()


1792
the complete dataset size:  (1792, 15, 128, 128)


### Extracting further information regarding the data set
***
This section shows how one can extract images of each defect from the larger scans 

In [55]:
#everything below here is for individual defects and stats

defects_images = [[],[],[],[],[],[],[],[],[],[],[],[],[]]
DB = []
As2 = []
dihydride = []
single_dihydride = []
onebyone = []
threebyone = []
dot = []
raised_silicon = []
etch_pit = []
gunk = []
step_edge = []
missing_dimer = []
unknown = []


for i in range(0,all_defects.shape[0]):#number of json files
    for j in range(1,all_defects.shape[1]):#different defects per json file, note we are skipping the image here
        #take the distance transpose to create peaks at the centre of the defects
        defect_ecl = distance_transform_edt(all_defects[i][j])*100
        defect_ecl = gaussian_filter(defect_ecl,sigma = (5,5),order = 0) # apply a gaussian filter to smooth the transform
        #find the peaks and output them as array
        peaks = np.transpose(np.where(detect_peaks(defect_ecl))) # creates an array with coordinates to each peak corresponding to each defect


        #filter peaks to remove ones that are from the same defect
        true_peaks = np.asarray(filter_quadratic(peaks,the_condition))

        for peak in range(0,true_peaks.shape[0]):
            x_peak = true_peaks[peak][0]
            y_peak = true_peaks[peak][1]
            defect_size = 20 # adjust this if you want to vary the dimension (in pixels) of the defect cutout.
            image_x = 1024
            image_y = 1024
            
            # this is a filter to remove any peaks from defects that sit close to the edge of the larger images.
            if defect_size < x_peak < image_x -defect_size and defect_size < y_peak < image_y -defect_size:
                image = all_defects[i][0] #index
                
                ###################################################################
                #Use this section if you want pure defects (defects which contain only one tip of defect)
                #This section finds pure defects (no other defects in image cut)
                other_defects = 0
                for z in range(1,all_defects.shape[1]):
                    other_defects = np.sum(all_defects[i][z][x_peak-defect_size:x_peak+defect_size,y_peak-defect_size:y_peak+defect_size])+other_defects
                    #print(z,": ",other_defects)
                defect = np.sum(all_defects[i][j][x_peak-defect_size:x_peak+defect_size,y_peak-defect_size:y_peak+defect_size])
                #print('defect',": ", defect)
                if other_defects-defect == 0:
                    defect = image[x_peak-defect_size:x_peak+defect_size,y_peak-defect_size:y_peak+defect_size]
                    defect = normalize(defect)
                    defects_images[j-1].append(defect)
                    #cv2.imwrite("defect_images_test\\" + str(i) + "_" + str(j) + "_pure.jpg",defect)
                else:
                    pass
                #end of single defect section
                #####################################################################
                """
                ####################################################################
                #use this section if you don't care if the image contains more than one type of defect
                defect = image[x_peak-defect_size:x_peak+defect_size,y_peak-defect_size:y_peak+defect_size]
                defect = normalize(defect)
                defects_images[j-1].append(defect)
                cv2.imwrite("defect_images_test\\" + str(i) + "_" + str(j) + ".jpg",defect)
                #end of multiple defect section
                #####################################################################
                """
            else: # skips the defect if it cuts into the edge of the image
                pass

# to create a set of h5 files for each defect type.
#for x in range(0,len(defects_images)):
    #h5f = h5py.File('single_defects_pure\\' + defect_classes[x] + '_pure.h5','w')
    #h5f.create_dataset(defect_classes[x],data = defects_images[x])
    #h5f.close()


#plt.imshow(defects[0]*255)
#plt.show()

#The following are calculated directly from teh JSON files above
print('defect_count: ', defects_count) #prints the number of defects for each class.
print('defect_pixels:', defects_pixels) # prints the number of pixels that correspond to each image.


defect_count:  [ 83  17 211  24  42  14 277 167  76  26   1 102  96]
defect_pixels: [48436 10336 46559  5392 13974  6756 33992 32071 23975 12668 24178 21166
 33681]
